# INTELIGENCIA ARTIFICIAL - EVALUACIÓN 1
Integrantes: Sebastian Racedo y Sebastian Valle

In [1]:
# Libraries Import
import math
import pandas as pd
import numpy as np

In [2]:
class Node:
    def __init__(self, value):
        self.children = []
        self.value = value
        
    def addNode(node):
        self.children.push(node)

In [3]:
# Read and split data into training and test sets
def SplitData(Raw_Data):
    data = pd.read_csv(Raw_Data,sep = ',')
    [n,m] = data.shape
    headers = list(data.columns.values)
    X = data.iloc[:,0:m-1]
    y = data.iloc[:,m-1]
    return data, X, y

In [4]:
def Entropy(data):
    Size_data = len(data)
    if Size_data == 0:
        return 0
    num_classes = [0 for i in data.iloc[:,-1].dropna().value_counts()]
    num_classes = data.iloc[:,-1].dropna().value_counts().tolist()
    num_classes = [x/Size_data for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*math.log(num,2)
    return ent*-1

## GainRatio DEBE aceptar valores numericos o NO numericos, Actualmente solo sirve para NO numericos

In [5]:
def Gain_Ratio(data,headers):
    List_entropy = []
    List_Sum= []
    List_split = []
    Global_Entropy = Entropy(data)
    print("Global entropy = ",Global_Entropy)
    for attribute in headers[0:len(headers)-1]:
        if (np.issubdtype(data[attribute].dtype, np.number) == False):
            entropy = []
            Split = 0
            for value in data[attribute].value_counts().index:
                # Gain Calculation
                Neg_branch = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'No')]
                Pos_branch  = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'Yes')]
                Total_branch = [Neg_branch,Pos_branch]
                Total_branch = pd.concat(Total_branch)
                probability_num = (len(Total_branch)/len(data))
                entropy.append(probability_num*Entropy(Total_branch))
                Split += probability_num*math.log(probability_num,2)
            Split = Split*-1
            List_split.append(Split)
            Sum_per_branch = sum(entropy)
            List_entropy.append(list(entropy))
            List_Sum.append(Sum_per_branch)
        else:
            pass
            #print(data.sort_values(by = [attribute]))
    Gain = np.subtract(Global_Entropy,List_Sum)
    GainRatio = np.divide(Gain,List_split)
    return GainRatio

In [6]:
def CalculateNode(data):
    if len(data.unique()) == 1:
        return Node(data.unique()[0])
    # TODO, base case 2

In [7]:
data, X, y = SplitData("Golf_data.csv")
headers = list(data.columns.values)
print(headers)

['Outlook', 'Temp', 'Humidity', 'Play Golf']


In [8]:
GainRatio = Gain_Ratio(data,headers)
print(GainRatio)

Global entropy =  0.863120568566631
[0.11810628 0.07467011 0.00606734]


In [9]:
GainRatio.max()

0.1181062751466107

In [10]:
data

,Outlook,Temp,Humidity,Play Golf
0,Rainy,Hot,Normal,No
1,Sunny,Hot,High,No
2,Overcast,Hot,Normal,Yes
3,Rainy,Hot,Normal,Yes
4,Rainy,Cool,Normal,Yes
5,Rainy,Cool,High,No
6,Overcast,Cool,High,Yes
7,Sunny,Hot,Normal,No
8,Sunny,Cool,Normal,Yes
9,Rainy,Cool,Normal,Yes
